#Basic of Chains in LangChain

# Install Libraries

In [ ]:
!pip install --upgrade transformers accelerate langchain-huggingface
!pip install -U bitsandbytes

# Import Libraries

In [ ]:
import torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

In [ ]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser

In [ ]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

# Import Model

## Download Model

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
model_id = "unsloth/gemma-3-1b-it"
# model_id = "Qwen/Qwen3-4B-Instruct-2507"

tokenizer = AutoTokenizer.from_pretrained(model_id)
raw_model_llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.float16 if torch.cuda.is_available() else 'auto',
    device_map="auto" ,
    offload_folder="offload",
    offload_state_dict=True,
    quantization_config = None
)


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

## Make Pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model=raw_model_llm,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.0
)


Device set to use cpu


In [ ]:
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={
        "max_new_tokens": 200,
        "temperature": 0.0
    }
)

## Make ChatModel

In [ ]:
model = ChatHuggingFace(llm=llm)

In [ ]:
llm.invoke('What is the capital city of bangladesh')

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'What is the capital city of bangladesh?\n\nThe capital city of Bangladesh is Dhaka.\n\nFinal Answer: The final answer is $\\boxed{Dhaka}$'

In [ ]:
model.invoke('What is the capital city of bangladesh')

AIMessage(content='<bos><start_of_turn>user\nWhat is the capital city of bangladesh<end_of_turn>\n<start_of_turn>model\nThe capital city of Bangladesh is **Dhaka**.\n', additional_kwargs={}, response_metadata={}, id='lc_run--019b106d-64e8-78a1-a8fb-ba5d410fb275-0')

## Chat Prompt Template

In [ ]:
gemma_template = """
<start_of_turn>user
{input}
<end_of_turn>
<start_of_turn>model
"""

template_prompt = ChatPromptTemplate.from_template(gemma_template)
chain = template_prompt | model

In [ ]:
chain.invoke({"input": "What is the capital city of Bangladesh?"}).content

'<bos><start_of_turn>user\n<start_of_turn>user\nWhat is the capital city of Bangladesh?\n<end_of_turn>\n<start_of_turn>model<end_of_turn>\n<start_of_turn>model\nThe capital city of Bangladesh is **Dhaka**. \n\nIt’s a bustling and historically significant city! 😊\n'

In [ ]:
Xllm = HuggingFacePipeline.from_model_id(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task='text-generation',
    pipeline_kwargs=dict(
        temperature=0.5,
        max_new_tokens=100
    )
)
model2 = ChatHuggingFace(llm=Xllm)

Device set to use cpu


In [ ]:
model2.invoke("What is the capital city of bangladesh?").content

'<|user|>\nWhat is the capital city of bangladesh?</s>\n<|assistant|>\nThe capital city of Bangladesh is Dhaka.'

# Simple Chain

In [ ]:
simple_prompts = PromptTemplate(
    template = """
      What is the capital city of {country}?
    """,
    input_variables = ['country']
)

In [ ]:
str_parser = StrOutputParser()

In [ ]:
simple_chain = simple_prompts | model | str_parser
# simple_result = simple_chain.invoke({"input": "What is the capital city of Bangladesh?"})
simple_result = simple_chain.invoke({"country": "Bangladesh"})

simple_result

'<bos><start_of_turn>user\nWhat is the capital city of Bangladesh?<end_of_turn>\n<start_of_turn>model\nThe capital city of Bangladesh is **Dhaka**.\n'

# Sequential Chains

In [ ]:
parser = StrOutputParser()

In [ ]:
template1 = PromptTemplate(
    template = "What is the Capital city of {country}?",
    input_variables=['country']
)

template2 = PromptTemplate(
    template = "Tell me 3 lines of poem for following text \n{capital}",
    input_variables = ['capital']
)

In [ ]:
seq_chain = template1 | model | parser | template2 | model | parser
seq_result = seq_chain.invoke({'country': 'bangladesh'})
seq_result

'<bos><start_of_turn>user\nTell me 3 lines of poem for following text \n<bos><start_of_turn>user\nWhat is the Capital city of bangladesh?<end_of_turn>\n<start_of_turn>model\nThe capital city of Bangladesh is **Dhaka**.<end_of_turn>\n<start_of_turn>model\nOkay, here are three lines of poetry inspired by that information:\n\nA city of ancient grace,\nDhaka stands, a vibrant space,\nWhere history and future embrace.'

In [ ]:
seq_result.split('<start_of_turn>')[-1]

'model\nOkay, here are three lines of poetry inspired by that information:\n\nA city of ancient grace,\nDhaka stands, a vibrant space,\nWhere history and future embrace.'

# Parallel Chains

## Prompt Template Create

In [ ]:
# Parallel 1, Make Summary
template1 = PromptTemplate(
    template = "lets summary of the following texts \n{text}",
    input_variables = ['text']
)

# Parallel 2, Make Question
template2 = PromptTemplate(
    template = "lets make 5 short question on following texts \n{text}",
    input_variables = ['text']
)

# Merge of Parallel 1 and Parallel 2
template3 = PromptTemplate(
    template = "Lets make merge these two answer on this format \n Answer1: Summary\n{summary}\nAnswer2: Questions:\n{questions}",
    input_variables = ['summary', 'questions']
)


## Parser

In [ ]:
parser = StrOutputParser()

## Make Parallel Runnable

In [ ]:
parallel_chain = RunnableParallel({
    'summary': template1 | model | parser,
    'questions': template2 | model2 | parser
})

## Merge Chains

In [ ]:
merge_chain = template3 | model | parser

## Final Chain

In [ ]:
final_chain = parallel_chain | merge_chain

## Invoke Chains

In [ ]:
with open('data.txt') as f:
    text = f.read()

In [ ]:
text

'Integrating Hugging Face models with LangChain allows you to leverage thousands of open-source models for a wide range of tasks, from text generation to embeddings. A dedicated langchain-huggingface package, jointly maintained by both teams, facilitates this integration. \nMethods of Integration\nYou can integrate Hugging Face models with LangChain in two primary ways:\n1. Using the Hugging Face Inference API (Cloud-Hosted)\nThis method uses the Hugging Face API to run models on serverless infrastructure, requiring an API token. \nSetup:\nCreate a free Hugging Face account and get a read-only API token from your profile settings.\nSet the token as an environment variable: os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_...".\nInstall the necessary library: pip install langchain-huggingface'

In [ ]:
final_result = final_chain.invoke({'text': text})
final_result

'<bos><start_of_turn>user\nLets make merge these two answer on this format \n Answer1: Summary\n<bos><start_of_turn>user\nlets summary of the following texts \nIntegrating Hugging Face models with LangChain allows you to leverage thousands of open-source models for a wide range of tasks, from text generation to embeddings. A dedicated langchain-huggingface package, jointly maintained by both teams, facilitates this integration. \nMethods of Integration\nYou can integrate Hugging Face models with LangChain in two primary ways:\n1. Using the Hugging Face Inference API (Cloud-Hosted)\nThis method uses the Hugging Face API to run models on serverless infrastructure, requiring an API token. \nSetup:\nCreate a free Hugging Face account and get a read-only API token from your profile settings.\nSet the token as an environment variable: os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_...".\nInstall the necessary library: pip install langchain-huggingface<end_of_turn>\n<start_of_turn>model\nOkay, 

# Conditional Chains

## Make Base Class

In [ ]:
class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')


## Make Parser

In [ ]:
parser = StrOutputParser()
pydantic_parser = PydanticOutputParser(pydantic_object=Feedback)

## Make Template

In [ ]:
pyd_template1 = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment classifier. Classify the sentiment of the provided feedback as either 'positive' or 'negative'. Your response MUST be a JSON object with a single key 'sentiment' and its corresponding value. Example output: {{\"sentiment\": \"positive\"}}\n"),
    ("human", "{feedback}"),
])

pyd_template1 = prompt1.partial(format_instructions = pydantic_parser.get_format_instructions())

## Classifier Chain

In [ ]:
classifier_chain = pyd_template1 | model | pydantic_parser

In [ ]:
result1 = classifier_chain.invoke({'feedback': 'This phone is very bad'})
result1

Feedback(sentiment='negative')

In [ ]:
type(result1)

__main__.Feedback

In [ ]:
result1.sentiment

'negative'

## Prompt2 - Template

In [ ]:
template2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

## Prompt3 - Template

In [ ]:
template3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

##  Branch Chain

In [ ]:
branch_chain = RunnableBranch(
    # (condition, chain)
    # if x == positive
    (lambda x: x.sentiment == 'positive', template2 | model | parser),

    # else if x == 'negative'
    (lambda x: x.sentiment == 'negative', template3 | model | parser),

    # else
    RunnableLambda(lambda x: "can not give answer here")
)

In [ ]:
cond_final_chain = classifier_chain | branch_chain

cond_result = cond_final_chain.invoke({'feedback': 'This is a good phone'})
cond_result

"<bos><start_of_turn>user\nWrite an appropriate response to this positive feedback \n sentiment='positive'<end_of_turn>\n<start_of_turn>model\nPlease provide the positive feedback you’d like me to respond to! I need the text of the feedback to be able to craft an appropriate response. 😊 \n\nOnce you paste it in, I’ll do my best to give you a thoughtful and helpful reply."

## Alternative Output Parser

In [ ]:
json_parser = JsonOutputParser(
    schema={
        "sentiment": {"type": "string", "enum": ["positive", "negative"]}
    }
)

prompt_template = PromptTemplate(
    template="""
You are a sentiment classifier.
Classify the sentiment of the following feedback:

{feedback}

Respond **ONLY** in valid JSON. Example:
{{"sentiment": "positive"}}
Do NOT include any explanations, extra examples, or markdown.
""",
    input_variables=["feedback"]
)

# Build chain
chain = prompt_template | llm | json_parser

# Run it
feedback_input = "This phone is very good"
output_json = chain.invoke({"feedback": feedback_input})

print(output_json)

{'sentiment': 'positive'}
